In [1]:
# Import regular expressions package.
import re

# convenient for making HTTP requests
import requests as rq

# dates and times:
import datetime as dt

# data frames 
import pandas as pd

#need to use this for downlaong the excel file
import urllib.request as urlrq


# CAO 2021 POINTS ANALYSIS

http://www2.cao.ie/points/l8.php


<br>

## Save original Data 

***


In [2]:
# docs.python.org/3/library/datetime.html
#now current date and time
now=dt.datetime.now()

In [3]:
now

datetime.datetime(2021, 11, 28, 23, 22, 30, 491351)

In [4]:
#create a path for the original data
now=dt.datetime.now()
nowstr = now.strftime('%Y%m%d_%H%M%S')
path2021 = "data/cao2021_" + nowstr + ".csv"

In [5]:
# fetch the CAO points url
resp = rq.get('http://www2.cao.ie/points/l8.php')
# have a quick peek.
resp

<Response [200]>

In [6]:
#the server uses the wrong encoding 
original_encoding =resp.encoding
#change to 
resp.encoding = "cp1252"

In [7]:
# save the original html file 
with open (path2021, "w") as f:
    f.write(resp.text)

In [8]:
# compile regular expression for matching lines
re_course = re.compile (r'([A-Z]{2}[0-9]{3})(.*)')

In [9]:
no_lines = 0
with open (path2021, "w") as f:
    # write a header row
    f.write(','.join(['code','course','pointsR1','pointsR2']) + '\n')
    for line in resp.iter_lines():
        dline = line.decode('cp1252')
        if re_course.fullmatch(dline):
            no_lines = no_lines + 1
            #the course code usinf [:5] here to pull the first 5 characters of the dataset which will be the course code ie AL321
            course_code = dline [:5]
            #print(course_code)# if we inpect the csv dataset we can see that the title starts at point number 7 and for the longest course title will stop at 56
            course_title= dline[7:57].strip()
            #print(course_title)
            #round 1 points:
            # split the line here further using one or more white spaces
            course_points=re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            #join the fields using a comma.
            linesplit = [course_code,course_title,course_points[0],course_points[1]]
            #rejoin the substrings with commas in between
            f.write(','.join(linesplit)+'\n')
#                print(f"'{course_code} {course_points}'")
print(f"Total numner of lines is {no_lines}.")  


Total numner of lines is 949.


In [10]:
df2021 = pd.read_csv(path2021)
df2021


,code,course,pointsR1,pointsR2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


<br>

# CAO 2020 points

http://www.cao.ie/index.php?page=points&p=2020

In [11]:
#create a path for the original data
path2020 = "data/cao2020_" + nowstr + ".xlsx"


In [12]:
# save original excel sheet url to disk.
urlrq.urlretrieve('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', path2020)

('data/cao2020_20211128_232230.xlsx',
 <http.client.HTTPMessage at 0x234c3f2ad00>)

### Load df using pandas

In [13]:
# Download and sparse the excel spreadsheet
#ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
df2020 = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', skiprows=10)

In [14]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# check the the excel sheet read is is correct 
df2020.iloc[354]

CATEGORY (i.e.ISCED description)    Information and Communication Technologies (ICTs)
COURSE TITLE                                                     Enterprise Computing
COURSE CODE2                                                                    DC120
R1 POINTS                                                                         357
R1 Random *                                                                       NaN
R2 POINTS                                                                         NaN
R2 Random*                                                                        NaN
EOS                                                                               357
EOS Random *                                                                      NaN
EOS Mid-point                                                                     392
LEVEL                                                                               8
HEI                                                   

In [16]:
df2020.iloc[1463]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [17]:
df2020.iloc[-3]

CATEGORY (i.e.ISCED description)    Information and Communication Technologies (ICTs)
COURSE TITLE                                                       Creative Computing
COURSE CODE2                                                                    WD211
R1 POINTS                                                                         271
R1 Random *                                                                       NaN
R2 POINTS                                                                         NaN
R2 Random*                                                                        NaN
EOS                                                                               271
EOS Random *                                                                      NaN
EOS Mid-point                                                                     318
LEVEL                                                                               8
HEI                                                 Wa

In [18]:
# create a file path for the pandas data
path3 = "data/cao2020_" + nowstr + ".csv"

In [19]:
df2020.to_csv(path3)

<br>

# CAO 2019 POINTS

***

http://www2.cao.ie/points/lvl8_19.pdf

### Steps to reproduce

- Download original pdf file.
- Open original pdf file in microsoft word.
- Save Mircosoft Word's converted pdf in the docx format.
- Resave word document for editing.
- Delete headers and footers.
- Delete preample in page 1.
- Select all and copy to notepad++.
- Delete blank lines and remove HEI headings and paste onto each course line respectively (using alt + scroll down).
- Insert double quotes along the HEI title as this will make it easier to edit using regular expressions. 
- Change backtick's to apostrophes.
- Go to repleace all with \t (tab delimiter) and replace with comma(,) as it is a CSV file.
- Many commas throughout the document so we couldn't load the dataframe as a csv file. Instead we went back to original format with the delimiter as the tab.
- When reading data in the sep = '\t' was used as the delimiter for separating the columns.
- Errors occured while laoding this data because unknowningly there were some double tabs on the document. Fix was to replace \t\t with \t. DF was able to be read in after this.
- I went back to the dataset and then replaced all ` back ticks ` with the correct ' forward tick mark.


In [20]:
df2019 = pd.read_csv('data/cao2019_20211129_edited.csv', sep = '\t')

In [21]:
df2019

,HEI,Course Code,INSTITUTION and COURSE,EOS,Mid
0,Athlone Institute of Technology,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,Athlone Institute of Technology,AL802,Software Design with Cloud Computing,301,306.0
2,Athlone Institute of Technology,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,Athlone Institute of Technology,AL805,Network Management and Cloud Infrastructure,329,442.0
4,Athlone Institute of Technology,AL810,Quantity Surveying,307,349.0
...,...,...,...,...,...
927,Waterford Institute of Technology,WD200,Arts (options),221,296.0
928,Waterford Institute of Technology,WD210,Software Systems Development,271,329.0
929,Waterford Institute of Technology,WD211,Creative Computing,275,322.0
930,Waterford Institute of Technology,WD212,Recreation and Sport Management,274,311.0


## Con cat and join:

In [22]:
courses2019 = df2019[['Course Code', 'INSTITUTION and COURSE']]
courses2019

,Course Code,INSTITUTION and COURSE
0,AL801,Software Design with Virtual Reality and Gaming
1,AL802,Software Design with Cloud Computing
2,AL803,Software Design with Mobile Apps and Connected...
3,AL805,Network Management and Cloud Infrastructure
4,AL810,Quantity Surveying
...,...,...
927,WD200,Arts (options)
928,WD210,Software Systems Development
929,WD211,Creative Computing
930,WD212,Recreation and Sport Management


In [23]:
courses2020 = df2020[['COURSE CODE2','COURSE TITLE']]
courses2020.columns = ['code','title']
courses2020

,code,title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,"First Year Art & Design (Common Entry,portfolio)"
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
1459,WD208,Manufacturing Engineering
1460,WD210,Software Systems Development
1461,WD211,Creative Computing
1462,WD212,Recreation and Sport Management


In [24]:
courses2021= df2021[['code','course']]
courses2021.columns= ['code','title']
courses2021

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
944,WD211,Creative Computing
945,WD212,Recreation and Sport Management
946,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [25]:
allcourses = pd.concat([courses2021,courses2020], ignore_index=True)
allcourses

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
2408,WD208,Manufacturing Engineering
2409,WD210,Software Systems Development
2410,WD211,Creative Computing
2411,WD212,Recreation and Sport Management


In [26]:
#when i sort by code on the
allcourses.sort_values('code')

,code,title
175,AC120,International Business
949,AC120,International Business
950,AC137,Liberal Arts
176,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
...,...,...
2411,WD212,Recreation and Sport Management
2412,WD230,Mechanical and Manufacturing Engineering
946,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [27]:
#pandas duplicate not unique :-). Finds all extra copies of courses.
# this will only return back the second copy of the duplicated course. 
allcourses[allcourses.duplicated()]

,code,title
949,AC120,International Business
950,AC137,Liberal Arts
952,AD102,Graphic Design and Moving Image Design (portfo...
955,AD204,Fine Art (portfolio)
956,AD211,Fashion Design (portfolio)
...,...,...
2404,WD200,Arts (options)
2409,WD210,Software Systems Development
2410,WD211,Creative Computing
2411,WD212,Recreation and Sport Management


In [28]:
#returns df with duplicates removed. 
allcourses.drop_duplicates()

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
2403,WD195,Architectural and Building Information Modelli...
2405,WD205,Molecular Biology with Biopharmaceutical Science
2406,WD206,Electronic Engineering
2407,WD207,Mechanical Engineering


In [29]:
allcourses[allcourses.duplicated(subset=['code'])]

,code,title
949,AC120,International Business
950,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
952,AD102,Graphic Design and Moving Image Design (portfo...
953,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
2404,WD200,Arts (options)
2409,WD210,Software Systems Development
2410,WD211,Creative Computing
2411,WD212,Recreation and Sport Management


In [30]:
#returns df with duplicates removed based only on code because courses may have been named differently over the years.
allcourses.drop_duplicates(subset=['code'], inplace=True, ignore_index=True)

In [31]:
allcourses

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
1512,WD188,Applied Health Care
1513,WD205,Molecular Biology with Biopharmaceutical Science
1514,WD206,Electronic Engineering
1515,WD207,Mechanical Engineering


### Join to the points

In [32]:
#df2021.set_index('code', inplace=True)
#df2021.columns = ['title', 'points_r1_2021', 'points_r2_2021']
# I had the code written here in a lower case c and this cell would not run. The devil is in the detail got "None of ['code'] are in the columns" as the error message 


In [33]:
df2021.set_index('code', inplace=True)
df2021.columns = ['title', 'points_r1_2021', 'points_r2_2021']
df2021

,title,points_r1_2021,points_r2_2021
code,,,
AL801,Software Design for Virtual Reality and Gaming,300,NaN
AL802,Software Design in Artificial Intelligence for...,313,NaN
AL803,Software Design for Mobile Apps and Connected ...,350,NaN
AL805,Computer Engineering for Network Infrastructure,321,NaN
AL810,Quantity Surveying,328,NaN
...,...,...,...
WD211,Creative Computing,270,NaN
WD212,Recreation and Sport Management,262,NaN
WD230,Mechanical and Manufacturing Engineering,230,230


In [46]:
# set the index to the code column:
allcourses.set_index('code', inplace =True)

KeyError: "None of ['code'] are in the columns"

In [47]:
allcourses= allcourses.join(df2021[['points_r1_2021']])
allcourses

ValueError: columns overlap but no suffix specified: Index(['points_r1_2021'], dtype='object')

In [35]:
# set the index to the code column:
allcourses.set_index('code', inplace =True)
allcourses

,title,points_r1_2021
code,,
AL801,Software Design for Virtual Reality and Gaming,NaN
AL802,Software Design in Artificial Intelligence for...,NaN
AL803,Software Design for Mobile Apps and Connected ...,NaN
AL805,Computer Engineering for Network Infrastructure,NaN
AL810,Quantity Surveying,NaN
...,...,...
WD188,Applied Health Care,NaN
WD205,Molecular Biology with Biopharmaceutical Science,NaN
WD206,Electronic Engineering,NaN


In [37]:
#allcourses.set_index('code').join(df2021[['code','pointsR1']].set_index('code'))


In [38]:
df2020_r1 = df2020[['COURSE CODE2','R1 POINTS']]
df2020_r1.columns = ['code','points_r1_2020']
df2020_r1

,code,points_r1_2020
0,AC120,209
1,AC137,252
2,AD101,#+matric
3,AD102,#+matric
4,AD103,#+matric
...,...,...
1459,WD208,188
1460,WD210,279
1461,WD211,271
1462,WD212,270


In [39]:
# Set the index to the code column.
df2020_r1.set_index('code', inplace=True)
df2020_r1

,points_r1_2020
code,
AC120,209
AC137,252
AD101,#+matric
AD102,#+matric
AD103,#+matric
...,...
WD208,188
WD210,279
WD211,271


In [40]:
# Join 2020 points to allcourses.
allcourses = allcourses.join(df2020_r1)
allcourses


,title,points_r1_2021,points_r1_2020
code,,,
AL801,Software Design for Virtual Reality and Gaming,NaN,303
AL802,Software Design in Artificial Intelligence for...,NaN,332
AL803,Software Design for Mobile Apps and Connected ...,NaN,337
AL805,Computer Engineering for Network Infrastructure,NaN,333
AL810,Quantity Surveying,NaN,319
...,...,...,...
WD188,Applied Health Care,NaN,201
WD205,Molecular Biology with Biopharmaceutical Science,NaN,228
WD206,Electronic Engineering,NaN,179
